#Create the environment

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/ESoWC

/content/drive/My Drive/ESoWC


In [3]:
import pandas as pd
import xarray as xr
import numpy as np
import random

from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import ExtraTreesRegressor

from sklearn.feature_selection import VarianceThreshold

from sklearn.decomposition import PCA

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

import matplotlib.pyplot as plt

#Load dataset and clean

In [4]:
dataset = xr.open_dataset('Data/3_months_dataset_complete_for_model_CO.nc')
dataset

<xarray.Dataset>
Dimensions:            (latitude: 33, longitude: 33, time: 2233)
Coordinates:
  * time               (time) datetime64[ns] 2019-05-01 ... 2020-05-31T23:00:00
  * latitude           (latitude) float64 43.0 43.25 43.5 ... 50.5 50.75 51.0
  * longitude          (longitude) float64 4.0 4.25 4.5 4.75 ... 11.5 11.75 12.0
Data variables: (12/32)
    EMISSIONS          (latitude, longitude, time) float64 ...
    u10                (latitude, longitude, time) float64 ...
    v10                (latitude, longitude, time) float64 ...
    hcc                (latitude, longitude, time) float64 ...
    lcc                (latitude, longitude, time) float64 ...
    tcc                (latitude, longitude, time) float64 ...
    ...                 ...
    CO2_tc_add_trend   (latitude, longitude, time) float64 ...
    CH4_tc_add_trend   (latitude, longitude, time) float64 ...
    NO2_tc_add_trend   (latitude, longitude, time) float64 ...
    CO_tc_add_trend    (latitude, longitude, time) float64 ...
    O3_tc_add_trend    (latitude, longitude, time) float64 ...
    traffic            (latitude, longitude, time) float64 ...

In [5]:
df = dataset.to_dataframe()
df = df.dropna()
df.head()

EMISSIONS  ...  traffic
latitude longitude time                               ...         
43.25    4.0       2019-05-01 12:00:00  6.484499e-13  ...      0.0
                   2019-05-01 13:00:00  6.373653e-13  ...      0.0
                   2019-05-01 14:00:00  6.484499e-13  ...      0.0
                   2019-05-01 15:00:00  6.318230e-13  ...      0.0
                   2019-05-01 16:00:00  6.429076e-13  ...      0.0

[5 rows x 32 columns]

In [6]:
df.shape

(1181625, 32)

#Managing dataframe

Now we create the feature days and hours

In [7]:
df_flat = df
df_flat['days'] = df_flat.index.get_level_values("time").day 
df_flat['hours'] = df_flat.index.get_level_values("time").hour
df_flat = df_flat.reset_index()
df_flat = df_flat.drop(["time"], axis=1)
df_flat.head()

,latitude,longitude,EMISSIONS,u10,v10,hcc,lcc,tcc,cvl,cvh,sp,tmp,sp_hum,rel_hum,tcw,tot_wind,tmp_shift_1,tot_wind_shift_12,rel_hum_shift_8,height,built,NO_tc,CO2_tc,CH4_tc,NO2_tc,CO_tc,O3_tc,NO_tc_add_trend,CO2_tc_add_trend,CH4_tc_add_trend,NO2_tc_add_trend,CO_tc_add_trend,O3_tc_add_trend,traffic,days,hours
0,43.25,4.0,6.484499e-13,0.285356,2.319711,0.018088,-2.980232e-08,0.018088,0.082098,0.25119,100360.708374,289.426036,0.007318,0.000194,12.639999,2.337196,289.225674,5.530020,0.000262,0.011574,0.00027,0.000002,412.511468,1876.872673,0.000002,0.001024,0.007330,0.000001,412.361549,1874.381777,0.000003,0.001050,0.007345,0.0,1,12
1,43.25,4.0,6.373653e-13,0.203581,2.883800,0.016387,3.089180e-03,0.019478,0.082098,0.25119,100329.526937,289.431417,0.007563,0.000192,12.728169,2.890977,289.426036,5.079804,0.000282,0.011574,0.00027,0.000002,412.511543,1875.634344,0.000002,0.001032,0.007318,0.000001,412.327826,1872.541453,0.000003,0.001036,0.007455,0.0,1,13
2,43.25,4.0,6.484499e-13,0.121806,3.447889,0.014686,6.178389e-03,0.020868,0.082098,0.25119,100298.345500,289.436798,0.007809,0.000190,12.816339,3.450040,289.431417,4.694267,0.000303,0.011574,0.00027,0.000001,412.511618,1874.396014,0.000002,0.001040,0.007305,0.000001,412.306684,1872.394813,0.000003,0.001038,0.007430,0.0,1,14
3,43.25,4.0,6.318230e-13,0.040032,4.011978,0.012986,9.267598e-03,0.022259,0.082098,0.25119,100267.164062,289.442179,0.008054,0.000188,12.904509,4.012178,289.436798,4.390482,0.000293,0.011574,0.00027,0.000001,412.511694,1873.157685,0.000003,0.001048,0.007293,0.000001,412.280245,1872.238919,0.000003,0.001039,0.007406,0.0,1,15
4,43.25,4.0,6.429076e-13,0.154884,4.080582,0.013983,1.355723e-02,0.068497,0.082098,0.25119,100265.157104,289.232368,0.008139,0.000186,13.140744,4.083521,289.442179,4.175291,0.000282,0.011574,0.00027,0.000001,412.511769,1871.919356,0.000003,0.001056,0.007280,0.000001,412.248509,1872.073772,0.000003,0.001041,0.007385,0.0,1,16


Split the dataframe in features and target

In [8]:
X = df_flat.drop(columns = ['EMISSIONS'])
y = df_flat['EMISSIONS']

input_variables = X.columns
target_variable = 'EMISSIONS'

seed = 1234

Normalizing

In [9]:
scaler = MinMaxScaler()
df_sc = scaler.fit_transform(df_flat)

df_norm = pd.DataFrame(df_sc, columns=df_flat.columns)
df_norm.head()

,latitude,longitude,EMISSIONS,u10,v10,hcc,lcc,tcc,cvl,cvh,sp,tmp,sp_hum,rel_hum,tcw,tot_wind,tmp_shift_1,tot_wind_shift_12,rel_hum_shift_8,height,built,NO_tc,CO2_tc,CH4_tc,NO2_tc,CO_tc,O3_tc,NO_tc_add_trend,CO2_tc_add_trend,CH4_tc_add_trend,NO2_tc_add_trend,CO_tc_add_trend,O3_tc_add_trend,traffic,days,hours
0,0.0,0.0,0.074988,0.482312,0.710230,0.018088,5.960425e-08,0.018088,0.102538,0.256059,0.895828,0.510123,0.340440,0.322168,0.211587,0.138425,0.506041,0.327561,0.514518,0.001282,0.001063,0.326037,0.860642,0.747803,0.064463,0.796936,0.182792,0.120147,0.877719,0.819828,0.137957,0.925361,0.167591,8.775587e-09,0.0,0.521739
1,0.0,0.0,0.074987,0.478491,0.733134,0.016387,3.089248e-03,0.019478,0.102538,0.256059,0.894531,0.510233,0.353058,0.316029,0.213451,0.171229,0.510123,0.300891,0.573612,0.001282,0.001063,0.315249,0.860654,0.737382,0.069884,0.811318,0.179467,0.119682,0.871018,0.799311,0.142278,0.897497,0.200375,8.775587e-09,0.0,0.565217
2,0.0,0.0,0.074988,0.474670,0.756037,0.014686,6.178437e-03,0.020868,0.102538,0.256059,0.893233,0.510343,0.365677,0.309889,0.215316,0.204347,0.510233,0.278053,0.632706,0.001282,0.001063,0.304460,0.860665,0.726962,0.075305,0.825701,0.176142,0.118838,0.866817,0.797676,0.138482,0.900819,0.192721,8.775587e-09,0.0,0.608696
3,0.0,0.0,0.074986,0.470849,0.778941,0.012986,9.267626e-03,0.022259,0.102538,0.256059,0.891936,0.510452,0.378295,0.303750,0.217181,0.237647,0.510343,0.260057,0.603268,0.001282,0.001063,0.293671,0.860677,0.716541,0.080726,0.840084,0.172816,0.117458,0.861563,0.795939,0.135205,0.903873,0.185701,8.775587e-09,0.0,0.652174
4,0.0,0.0,0.074988,0.476216,0.781727,0.013983,1.355723e-02,0.068497,0.102538,0.256059,0.891852,0.506177,0.382651,0.300012,0.222177,0.241873,0.510452,0.247309,0.573831,0.001282,0.001063,0.282882,0.860688,0.706121,0.086147,0.854467,0.169491,0.115542,0.855257,0.794097,0.132446,0.906659,0.179315,8.775587e-09,0.0,0.695652


#Baseline performance: Linear regression with 10 fold cross validation

In [10]:
kfolds = KFold(10,shuffle=True,random_state=seed)

model = linear_model.LinearRegression()

In [11]:
scores = cross_val_score(model, df_norm[input_variables], df_norm[target_variable], cv=kfolds)

print("R2 Mean %.3f StdDev %.3f"%(scores.mean(),scores.std()))

R2 Mean 0.150 StdDev 0.004


In [12]:
scores = cross_val_score(model, df_norm[input_variables], df_norm[target_variable], cv=kfolds, scoring='neg_mean_squared_error')

print("MSE %.3f StdDev %.3f"%(abs(scores.mean()),scores.std()))

MSE 0.000 StdDev 0.000
